In [ ]:
#get file from path

#read text from file

#make frequency dict

#build heap, using Frequency dict

#construct the binary tree

#construct the codes from the binary tree

#create encoded text using code

#we will have to pad our encoded text, first 8 bits represent the number of 0s added at the end

#put this encoded text into the binary file

#return binary file as output

In [1]:
import heapq
import os
class BinaryTreeNode :
    
    def __init__(self,value,frequency) :
        self.value = value
        self.frequency = frequency 
        self.right = None
        self.left = None
    
    def __lt__(self,other) : #overwriting the comparison operator, this will now compare the objects of the class based on their frequency
                             #here it is the less than operator. based on returned true and false it will arrange the values.
        return self.frequency < other.frequency
    
    # def __eq__(self,other) :
    #     return self.frequency == other.frequency

class HuffmanCoding :

    def __init__(self,path) :
        self.path = path
        self.__heap = []
        self.__codeMap = {} #code to the char map
        self.__reverseCodeMap = {} #char to code map

    def __makeFrequencyDict(self,text) :
        frequency_dict = {}
        for char in text :
            frequency_dict[char] = frequency_dict.get(char,0) + 1
        return frequency_dict

    def __createHeap(self,frequency_dict) :

        for key in frequency_dict :
            frequency = frequency_dict[key] 
            binary_tree_node = BinaryTreeNode(key,frequency)
            heapq.heappush(self.__heap,binary_tree_node)    

    def __createTree(self) :
        
        while len(self.__heap) > 1 :
            binaryTreeNode1 = heapq.heappop(self.__heap)
            binaryTreeNode2 = heapq.heappop(self.__heap)
            freq_sum = binaryTreeNode1.frequency + binaryTreeNode2.frequency
            newNode = BinaryTreeNode(None,freq_sum)
            newNode.left = binaryTreeNode1
            newNode.right = binaryTreeNode2
            heapq.heappush(self.__heap,newNode)

    def __buildCode(self) :
        root = heapq.heappop(self.__heap)
        self.__buildCodeHelper(root,'')

    def __buildCodeHelper(self,root,current_bits) :
        if root == None :
            return
        if root.value != None :
            self.__codeMap[root.value] = current_bits
            self.__reverseCodeMap[current_bits] = root.value
            return 

        self.__buildCodeHelper(root.left,current_bits+'0')
        self.__buildCodeHelper(root.right,current_bits+'1')
  
    def __getEncodedText(self,text) : #basically convert the text to the bit wise representation

        encodedText = ''
        for char in text :
            encodedText += self.__codeMap[char]
            #encodedText += ' '
        
        #print(encodedText)
        return encodedText  

    def __padding(self,encodedText) :
        paddedAmount = 8 - (len(encodedText)%8)
        for i in range(paddedAmount) :
            encodedText += '0'
        #print(paddedAmount)
        padding = '{0:08b}'.format(paddedAmount) #converts the padded amount into 8 bit number.
        padded_encoded_text = padding + encodedText #adding padding to the start of the number
        
        
        return padded_encoded_text

    def __getBytesArray(self,padded_encoded_text) :
        
        n = len(padded_encoded_text)
        bytesArr = []
        for i in range(0,n,8) :
            byte = padded_encoded_text[i : i + 8]
            bytesArr.append(int(byte,2))
        
        return bytesArr

    def compress(self) :
        filename,filetype = os.path.splitext(self.path) #split smaple.txt to sample and .txt
        output_path = filename + '.bin' #make sample into sample.bin file
        with open(self.path,'r+') as file,open(output_path,'wb') as output : #open the path file as read and write, and the output as write binary
            text = file.read()
            text = text.rstrip() #remove all the extra blank spaces

            frequency_Dict = self.__makeFrequencyDict(text) #make the frequency dictionary
            self.__createHeap(frequency_Dict) #create heap of the dictionary
            self.__createTree() #create the binary tree
            self.__buildCode() #create codes for all the alphabets
            encodedText = self.__getEncodedText(text) #make the encoded text according to the input string
            padded_encoded_text = self.__padding(encodedText) #add padding
            bytesArray = self.__getBytesArray(padded_encoded_text) #make subdivisions of length 8 and store decimal value in byte string
            final_bytes = bytes(bytesArray) #create final bytes using the byteArr 


            output.write(final_bytes) #write the final_byte in the output

        print('compressed')
        return output_path



    def __removePadding(self,bit_string) :
        paddedInformation = bit_string[:8]
        extrapadding = (int(paddedInformation,2))
        bit_string = bit_string[8:]
        actual_text = bit_string[:-1*extrapadding]
        
        return actual_text
    
    def __decompressText(self,text) :

        decodedText = ''
        current_bits = ''
        for bits in text :
            current_bits+=bits
            if current_bits in self.__reverseCodeMap :
                character = self.__reverseCodeMap[current_bits]
                decodedText += character
                current_bits = ''

        return decodedText


    def decompress(self,input_path) :
        filename,filetype = os.path.splitext(self.path)
        output_path = filename + '_decompressed' + '.txt'
        with open(input_path,'rb') as file, open (output_path,'w') as output : #input in read binary
            bit_string =''
            byte = file.read(1) #reading the file one by one, so basically reading 8 bits at once.
            while byte :
                byte = ord(byte) #get the decimal value for the byte
                bits = bin(byte)[2:].rjust(8,'0') #create bits from the byte number, it will return in the form of 'b101' for 3, so we will slice the string into 101 and using rjust make it 00000101
                bit_string += bits #add the bit to the bit_string, we will get padded_encoded_string as a result
                byte = file.read(1)
            #print(bit_string)
            actual_text = self.__removePadding(bit_string)
            decompressedText = self.__decompressText(actual_text)
            output.write(decompressedText)


   

In [4]:
path = r'C:\Users\Aditya Kalhan\OneDrive - Thapar University\College\Coding Ninja\DSA\codes\Milestone 3\Huffman Coding\sample.txt'
h = HuffmanCoding(path)
h.compress()   
reversePath =  r'C:\Users\Aditya Kalhan\OneDrive - Thapar University\College\Coding Ninja\DSA\codes\Milestone 3\Huffman Coding\sample.bin'
#h1 = HuffmanCoding(reversePath)
h.decompress(reversePath)

compressed
